In [ ]:
# execute this in command line on all machines to be used as workers before initiating the hyperparamer search 
# ! pip install -U trains-agent==0.15.0
# ! trains-agent daemon --queue default

# pip install with locked versions
! pip install -U pandas==1.0.3
! pip install -U trains==0.15.0
! pip install -U hpbandster==0.7.4  # Needed only for Bayesian optimization Hyper-Band

In [ ]:
from trains.automation import UniformParameterRange, UniformIntegerParameterRange
from trains.automation import RandomSearch, HyperParameterOptimizer
from trains.automation.hpbandster import OptimizerBOHB  # Needed only for Bayesian optimization Hyper-Band

from trains import Task

In [ ]:
task = Task.init(project_name='Hyper-Parameter Search', task_name='Hyper-Parameter Optimization')

In [ ]:
#####################################################################
### Don't forget to replace this default id with your own task id ###
#####################################################################
TEMPLATE_TASK_ID = 'd8e928460f98437c998f3597768597f8'

In [ ]:
optimizer = HyperParameterOptimizer(
    base_task_id=TEMPLATE_TASK_ID,  # This is the experiment we want to optimize
    # here we define the hyper-parameters to optimize
    hyper_parameters=[
        UniformIntegerParameterRange('number_of_epochs', min_value=5, max_value=15, step_size=1),
        UniformIntegerParameterRange('batch_size', min_value=2, max_value=12, step_size=2),
        UniformParameterRange('dropout', min_value=0, max_value=0.5, step_size=0.05),
        UniformParameterRange('base_lr', min_value=0.0005, max_value=0.01, step_size=0.0005),
    ],
    # this is the objective metric we want to maximize/minimize
    objective_metric_title='accuracy',
    objective_metric_series='total',
    objective_metric_sign='max',  # maximize or minimize the objective metric
    max_number_of_concurrent_tasks=3,  # number of concurrent experiments
    # setting optimizer - trains supports GridSearch, RandomSearch or OptimizerBOHB
    optimizer_class=OptimizerBOHB,  # can be replaced with OptimizerBOHB
    execution_queue='default',  # queue to schedule the experiments for execution
    optimization_time_limit=30.,  # time limit for each experiment (optional, ignored by OptimizerBOHB)
    pool_period_min=1,  # Check the experiments every x minutes
    # set the maximum number of experiments for the optimization.
    # OptimizerBOHB sets the total number of iteration as total_max_jobs * max_iteration_per_job
    total_max_jobs=12,
    # setting OptimizerBOHB configuration (ignored by other optimizers)
    min_iteration_per_job=15000,  # minimum number of iterations per experiment, till early stopping
    max_iteration_per_job=150000,  # maximum number of iterations per experiment
)

In [ ]:
optimizer.set_time_limit(in_minutes=120.0)  # set the time limit for the optimization process
optimizer.start()  
optimizer.wait()  # wait until process is done
optimizer.stop()  # make sure background optimization stopped

In [ ]:
# optimization is completed, print the top performing experiments id
k = 3
top_exp = optimizer.get_top_experiments(top_k=k)
print('Top {} experiments are:'.format(k))
for n, t in enumerate(top_exp, 1):
    print('Rank {}: task id={} |result={}'
          .format(n, t.id, t.get_last_scalar_metrics()['accuracy']['total']['last']))